In [4]:
from datasets import load_dataset
from transformers import AutoImageProcessor, AutoModelForImageClassification, TrainingArguments, Trainer, ViTImageProcessor

# Step 1: Load the dataset
dataset = load_dataset("cqin/three_phase-plant_health")  # Example dataset


C:\Users\Naya\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
from transformers import AutoImageProcessor, AutoModelForImageClassification, TrainingArguments, Trainer, ViTImageProcessor


In [7]:
# Step 2: Load the processor and model
model_name = "swueste/plant-health-image-classifier"
processor = ViTImageProcessor.from_pretrained(model_name)
model = AutoModelForImageClassification.from_pretrained(model_name, num_labels=3)

In [8]:
dataset = dataset['train'].train_test_split(test_size=0.3, seed=42)

In [9]:
def transform(example_batch):
    # Take a list of PIL images and turn them to pixel values
    inputs = processor([x for x in example_batch['image']], return_tensors='pt')

    inputs['label'] = example_batch['label']
    return inputs

In [10]:
prepared_ds = dataset.with_transform(transform)

In [11]:
import torch

def collate_fn(batch):
    return {
        'pixel_values': torch.stack([x['pixel_values'] for x in batch]),
        'labels': torch.tensor([x['label'] for x in batch])
    }

In [12]:
!pip install evaluate
import evaluate
import numpy as np
metric = evaluate.load("accuracy")
def compute_metrics(p):
    return metric.compute(predictions=np.argmax(p.predictions, axis=1), references=p.label_ids)


In [13]:
from transformers import ViTForImageClassification

labels = prepared_ds['train'].features['label'].names

model = ViTForImageClassification.from_pretrained(
    model_name,
    num_labels=len(labels),
    id2label={str(i): c for i, c in enumerate(labels)},
    label2id={c: str(i) for i, c in enumerate(labels)}
)

In [14]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir="./result",
  per_device_train_batch_size=16,
  evaluation_strategy="steps",
  num_train_epochs=3,
  fp16=True,
  save_steps=100,
  eval_steps=100,
  logging_steps=10,
  learning_rate=2e-4,
  save_total_limit=2,
  remove_unused_columns=False,
  push_to_hub=False,
  report_to='tensorboard',
  load_best_model_at_end=True,
)

C:\Users\Naya\AppData\Roaming\Python\Python310\site-packages\transformers\training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [15]:
train_ds = prepared_ds['train'].select(range(1000))
eval_ds = prepared_ds['test'].select(range(300))

In [17]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=collate_fn,
    compute_metrics=compute_metrics,
    train_dataset=train_ds,
    eval_dataset=eval_ds,
)

In [18]:
train_results = trainer.train()

Step,Training Loss,Validation Loss,Accuracy
100,0.012900,0.049762,0.990000


Validation Loss у исходной модели равно 0.260014